In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta


In [1]:

import pandas as pd
import random
from datetime import datetime, timedelta
import calendar # 월별 마지막 날짜 계산을 위해 추가

# ==========================================
# 1. 설정 및 리스트 정의 (기존과 동일)
# ==========================================
num_rows_per_month = 130  # 매달 생성할 목표 데이터 수
year = 2026

category_map_random = {
    '식비': ['스타벅스', '맥도날드', '김밥천국', '배달의민족', '이마트24', 'GS25', '투썸플레이스', '쿠팡이츠', '구내식당', '삼겹살집'],
    '교통비': ['지하철', '택시', '버스', 'SK주유소', '카카오T', '코레일', 'GS칼텍스'],
    '쇼핑': ['쿠팡', '다이소', '무신사', '올리브영', '네이버쇼핑', '유니클로', '무인양품', 'ZARA'],
    '의료/건강': ['약국', '내과', '헬스장할부', '치과', '정형외과'],
    '문화/여가': ['CGV', '교보문고', '롯데월드', 'PC방', '볼링장', '야구장'],
    '교육': ['토익접수', '교재구입', '인프런', '영어 강의'],
    '기타': ['모임회비', '경조사비', '더치페이']
}

housing_internet = ['월세', '관리비', 'SKT통신비', '도시가스', '인터넷요금']
subscription_items = ['넷플릭스', '유튜브프리미엄', '멜론', '쿠팡와우', 'ChatGPT']
payment_methods = ['신용카드', '체크카드', '계좌이체', '현금']

categories = list(category_map_random.keys())
weights = [0.37, 0.21, 0.17, 0.05, 0.14, 0.04, 0.02] 

# ==========================================
# 2. 로직 함수 (날짜 생성 로직 업그레이드)
# ==========================================

def get_random_date_str(year, month):
    """해당 월의 마지막 날짜를 계산해서 안전하게 랜덤 날짜 생성"""
    # monthrange: (시작요일, 그 달의 총 일수) 반환 -> 예: 2월은 28 반환
    _, last_day = calendar.monthrange(year, month)
    
    rand_day = random.randint(1, last_day)
    target_date = datetime(year, month, rand_day)

    # 날짜 포맷 랜덤 섞기 (데이터 더럽히기)
    r = random.random()
    if r < 0.8: return target_date.strftime("%Y-%m-%d")
    elif r < 0.9: return target_date.strftime("%Y.%m.%d")
    else: return target_date.strftime("%Y/%m/%d")

def modified_amount(category, fixed_value=None):
    """금액 생성 (기존 동일)"""
    r = random.random()
    
    if fixed_value is not None:
        base_amount = fixed_value
    else:
        if category == '교육': base_amount = random.randint(2, 10) * 10000 
        elif category == '쇼핑': base_amount = random.randint(5, 150) * 1000 
        elif category == '식비': base_amount = random.randint(4, 25) * 1000 
        elif category == '교통비': base_amount = random.randint(15, 250) * 100 
        elif category == '기타': base_amount = random.randint(3, 10) * 10000 
        elif category == '구독': base_amount = random.randint(10, 25) * 1000 
        else: base_amount = random.randint(5, 50) * 1000
        
    # 금액 포맷 랜덤 섞기 (콤마 넣기 등)
    if r < 0.6: return base_amount
    elif r < 0.9: return f"{base_amount:,}"
    else: return str(base_amount)

def determine_essential(category, desc):
    """필수 지출 판별 (기존 동일)"""
    if category in ['주거/통신', '의료/건강', '교통비']:
        if desc == '택시': return False 
        return True
    
    if category == '식비':
        if desc in ['구내식당', '김밥천국', '이마트24', 'GS25']: return True
        return False
        
    if category == '쇼핑':
        if desc == '다이소': return True 
        return False 
        
    if category == '교육': return True 

    return False

# ==========================================
# 3. 데이터 조립 (1월~12월 반복)
# ==========================================

data = []

# 1월부터 12월까지 루프
for month in range(1, 13):
    
    # -----------------------------------------------------
    # (1) 주거/통신 (매달 1회씩 발생)
    for item in housing_internet:
        if item == '월세': 
            price = 700000
            # 월세는 매달 25일로 고정 (단, 25일날 처리)
            fix_date = datetime(year, month, 25)
        elif item == '관리비': price = random.randint(10, 15) * 10000; fix_date = None
        elif item == 'SKT통신비': price = random.randint(5, 8) * 10000; fix_date = None
        elif item == '인터넷요금': price = random.randint(2, 3) * 10000; fix_date = None
        else: price = random.randint(1, 3) * 10000; fix_date = None
        
        # 날짜 포맷팅
        if fix_date:
            # 월세 날짜도 포맷 섞기 적용
            date_str = fix_date.strftime("%Y-%m-%d") 
        else:
            date_str = get_random_date_str(year, month)
            
        amount_val = modified_amount('주거/통신', fixed_value=price)
        data.append([date_str, amount_val, '주거/통신', item, '계좌이체', True, True]) 

    # -----------------------------------------------------
    # (2) 구독 (매달 1회씩 발생)
    for item in subscription_items:
        date_str = get_random_date_str(year, month)
        amount_val = modified_amount('구독')
        data.append([date_str, amount_val, '구독', item, '신용카드', True, False])

    # -----------------------------------------------------
    # (3) 랜덤 데이터 채우기
    # 이번 달 남은 칸 수 계산
    current_fixed_count = len(housing_internet) + len(subscription_items)
    remaining_rows = num_rows_per_month - current_fixed_count

    for _ in range(remaining_rows):
        cat = random.choices(categories, weights=weights, k=1)[0]
        desc = random.choice(category_map_random[cat])
            
        date_val = get_random_date_str(year, month)
        amount_val = modified_amount(cat)
        pay_method = random.choice(payment_methods)
        
        is_fixed = random.choice([True, False]) if cat == '교육' else False
        is_essential = determine_essential(cat, desc)
        if random.random() < 0.05: is_essential = not is_essential

        data.append([date_val, amount_val, cat, desc, pay_method, is_fixed, is_essential])

# 섞기 (날짜 순서가 뒤죽박죽인게 더 리얼하므로)
random.shuffle(data)

# DataFrame 생성
df = pd.DataFrame(data, columns=['date', 'amount', 'category', 'description', 'payment_method', 'is_fixed', 'essential'])

# ==========================================
# 4. 결과 저장
# ==========================================
filename = "expense_data_annual.csv"
df.to_csv(filename, index=False, encoding='utf-8-sig')

print(f"✅ 1년치 데이터 생성 완료! 총 데이터 개수: {len(df)}개")
print(f"파일 저장됨: {filename}")

# 미리보기 (날짜가 섞여있는지 확인)
print(df.head())

✅ 1년치 데이터 생성 완료! 총 데이터 개수: 1560개
파일 저장됨: expense_data_annual.csv
         date  amount category description payment_method  is_fixed  essential
0  2026/10/01  23,000    문화/여가         볼링장           신용카드     False      False
1  2026-03-15   65000       쇼핑        ZARA             현금     False      False
2  2026/08/07  17,000      교통비          버스           신용카드     False       True
3  2026-02-13  125000       쇼핑        무인양품           계좌이체     False       True
4  2026-05-03   60000       쇼핑        ZARA           체크카드     False      False
